In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, Model
from tensorflow.keras.applications import EfficientNetV2B0  # Use a lighter version
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical

# Load the CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Normalize the dataset
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# Resize images to 128x128 to reduce memory usage
x_train_resized = tf.image.resize(x_train, (128, 128))
x_test_resized = tf.image.resize(x_test, (128, 128))

# One-hot encode labels
y_train_categorical = to_categorical(y_train, 10)
y_test_categorical = to_categorical(y_test, 10)

# EfficientNetV2B0 Branch
input_efficientnet = layers.Input(shape=(128, 128, 3))
efficient_net = EfficientNetV2B0(include_top=False, weights='imagenet', input_shape=(128, 128, 3))
x_efficient = efficient_net(input_efficientnet)
x_efficient = layers.Flatten()(x_efficient)

# Dense Layers
x = layers.Dense(128, activation='relu')(x_efficient)
x = layers.Dense(64, activation='relu')(x)

# Output Layer
output = layers.Dense(10, activation='softmax')(x)

# Define the final model
model = Model(inputs=input_efficientnet, outputs=output)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Print model summary
model.summary()

# Create TensorFlow datasets for training and testing
train_dataset = tf.data.Dataset.from_tensor_slices((x_train_resized, y_train_categorical)).batch(8).prefetch(tf.data.AUTOTUNE)
test_dataset = tf.data.Dataset.from_tensor_slices((x_test_resized, y_test_categorical)).batch(8).prefetch(tf.data.AUTOTUNE)

# Train the model
model.fit(train_dataset, epochs=10, validation_data=test_dataset)

# Evaluate the model
test_loss, test_acc = model.evaluate(test_dataset, verbose=2)
print(f"Test Accuracy: {test_acc * 100:.2f}%")
